In [91]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time
from collections import deque
import pandas as pd

# === CONFIG ===
DATA_DIR = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\SignLanguageRecognition\dataset\FSL"
os.makedirs(DATA_DIR, exist_ok=True)

label = input("Enter label: ").strip().lower()
SAVE_PATH = os.path.join(DATA_DIR, f"{label}.csv")

SEQ_LEN = 50                  # frames per sequence
MAX_SEQUENCES = 500           # NEW — auto-stop when reached

# === MediaPipe Hands ===
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.6)

sequence = deque(maxlen=SEQ_LEN)
collected_sequences = []

cap = cv2.VideoCapture(0)
print("\n📷 Starting capture in 3 seconds...")
time.sleep(3)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Extract 126 features per frame
    left = np.zeros(63)
    right = np.zeros(63)

    # === NO HANDS DETECTED HANDLING (NEW) ===
    if not (results.multi_hand_landmarks and results.multi_handedness):
        cv2.putText(frame, "No hands detected!", (10, 35),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

        cv2.putText(frame, f"Sequences: {len(collected_sequences)}/{MAX_SEQUENCES}",
                    (10, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                    (255, 255, 255), 2)

        cv2.imshow("Dynamic Capture", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        continue  # Skip to next frame (NO saving)

    # If hands detected
    for i, hand in enumerate(results.multi_hand_landmarks):
        handed = results.multi_handedness[i].classification[0].label
        lm_list = []

        for lm in hand.landmark:
            lm_list.extend([lm.x, lm.y, lm.z])

        if handed.lower() == "left":
            left = np.array(lm_list)
        else:
            right = np.array(lm_list)

    # Combine features
    frame_features = np.concatenate([left, right])
    sequence.append(frame_features)

    # Draw
    for landmark in results.multi_hand_landmarks:
        mp_draw.draw_landmarks(frame, landmark, mp_hands.HAND_CONNECTIONS)

    cv2.putText(frame, f"Label: {label}", (10, 35),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.putText(frame, f"Frames: {len(sequence)}/{SEQ_LEN}", 
                (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    cv2.putText(frame, f"Sequences: {len(collected_sequences)}/{MAX_SEQUENCES}",
                (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    cv2.imshow("Dynamic Capture", frame)

    # === Save when full sequence ===
    if len(sequence) == SEQ_LEN:
        collected_sequences.append(np.array(sequence).flatten())
        print(f"📦 Saved sequence #{len(collected_sequences)}")

    # === AUTO CLOSE WHEN 500 REACHED (NEW) ===
    if len(collected_sequences) >= MAX_SEQUENCES:
        print("\n🎉 500 sequences collected! Auto-saving and closing...")
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# === SAVE DATA ===
if collected_sequences:
    df = pd.DataFrame(collected_sequences)
    df["label"] = label
    df.to_csv(SAVE_PATH, index=False)

    print(f"\n✅ Saved dataset to {SAVE_PATH}")
    print(f"Total sequences collected: {len(collected_sequences)}")
else:
    print("🚫 No sequences collected.")



📷 Starting capture in 3 seconds...
📦 Saved sequence #1
📦 Saved sequence #2
📦 Saved sequence #3
📦 Saved sequence #4
📦 Saved sequence #5
📦 Saved sequence #6
📦 Saved sequence #7
📦 Saved sequence #8
📦 Saved sequence #9
📦 Saved sequence #10
📦 Saved sequence #11
📦 Saved sequence #12
📦 Saved sequence #13
📦 Saved sequence #14
📦 Saved sequence #15
📦 Saved sequence #16
📦 Saved sequence #17
📦 Saved sequence #18
📦 Saved sequence #19
📦 Saved sequence #20
📦 Saved sequence #21
📦 Saved sequence #22
📦 Saved sequence #23
📦 Saved sequence #24
📦 Saved sequence #25
📦 Saved sequence #26
📦 Saved sequence #27
📦 Saved sequence #28
📦 Saved sequence #29
📦 Saved sequence #30
📦 Saved sequence #31
📦 Saved sequence #32
📦 Saved sequence #33
📦 Saved sequence #34
📦 Saved sequence #35
📦 Saved sequence #36
📦 Saved sequence #37
📦 Saved sequence #38
📦 Saved sequence #39
📦 Saved sequence #40
📦 Saved sequence #41
📦 Saved sequence #42
📦 Saved sequence #43
📦 Saved sequence #44
📦 Saved sequence #45
📦 Saved sequence #46
📦 Saved

In [92]:
import numpy as np
import pandas as pd
import glob
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# =============================================
#        📁 BASE PATHS
# =============================================
BASE_DIR = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\SignLanguageRecognition"
ASL_DIR = os.path.join(BASE_DIR, "dataset", "ASL")
FSL_DIR = os.path.join(BASE_DIR, "dataset", "FSL")
MODELS_DIR = os.path.join(BASE_DIR, "models")

os.makedirs(MODELS_DIR, exist_ok=True)


# =============================================
#        🔄 LOAD ASL + FSL WITHOUT PREFIXES
# =============================================
def load_dataset(folder_path):
    files = glob.glob(os.path.join(folder_path, "*.csv"))
    dfs = []
    for f in files:
        df = pd.read_csv(f)
        dfs.append(df)
    return dfs

asl_dfs = load_dataset(ASL_DIR)
fsl_dfs = load_dataset(FSL_DIR)

df = pd.concat(asl_dfs + fsl_dfs, ignore_index=True)

print(f"📌 Loaded ASL files: {len(asl_dfs)}")
print(f"📌 Loaded FSL files: {len(fsl_dfs)}")
print(f"📌 Total merged samples: {len(df)}")


# =============================================
#        🎯 SEPARATE LABELS + FEATURES
# =============================================
labels = df['label']
df = df.drop('label', axis=1)

SEQ_LEN = 50
FEATURES = 126

# Reshape dataset
X = df.values.reshape(len(df), SEQ_LEN, FEATURES)

# Encode labels
le = LabelEncoder()
y = le.fit_transform(labels)

print("\n📌 Combined Classes:", list(le.classes_))
print("📌 Total Classes:", len(le.classes_))

# Save the merged classes list
np.save(os.path.join(MODELS_DIR, "classes.npy"), le.classes_)


# =============================================
#        ✂️ TRAIN / TEST SPLIT
# =============================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)


# =============================================
#        💾 SAVE PROCESSED DATASETS TO ASL/FSL
# =============================================
def save_processed(folder, X_train, X_test, y_train, y_test, classes):
    processed = os.path.join(folder, "processed")
    os.makedirs(processed, exist_ok=True)

    np.save(os.path.join(processed, "X_train.npy"), X_train)
    np.save(os.path.join(processed, "X_test.npy"), X_test)
    np.save(os.path.join(processed, "y_train.npy"), y_train)
    np.save(os.path.join(processed, "y_test.npy"), y_test)
    np.save(os.path.join(processed, "classes.npy"), classes)

    print(f"💾 Saved processed dataset → {processed}")

save_processed(ASL_DIR, X_train, X_test, y_train, y_test, le.classes_)
save_processed(FSL_DIR, X_train, X_test, y_train, y_test, le.classes_)


# =============================================
#     🚀 TFLITE-FRIENDLY GRU MODEL
# =============================================
model = tf.keras.Sequential([

    tf.keras.layers.Conv1D(
        64, 3, padding='same', activation='relu',
        input_shape=(SEQ_LEN, FEATURES)
    ),

    tf.keras.layers.DepthwiseConv1D(3, padding='same', activation='relu'),

    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(64),

    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


# =============================================
#        🧠 TRAIN MODEL
# =============================================
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=40,
    batch_size=32
)


# =============================================
#        📊 EVALUATE MODEL
# =============================================
loss, acc = model.evaluate(X_test, y_test)
print("\n📊 Test Accuracy:", acc)


# =============================================
#        💾 SAVE MODELS
# =============================================
keras_path = os.path.join(MODELS_DIR, "Sign_Model.keras")
model.save(keras_path)
print("\n💾 Saved Keras model:", keras_path)


# =============================================
#        🚀 TFLITE CONVERSION
# =============================================
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

tflite_path = os.path.join(MODELS_DIR, "Sign_Model.tflite")
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print("\n💾 Saved TFLite model:", tflite_path)


📌 Loaded ASL files: 30
📌 Loaded FSL files: 30
📌 Total merged samples: 30000

📌 Combined Classes: ['ano', 'asul', 'bahay', 'bakit', 'basahin', 'black', 'blue', 'book', 'bukas', 'car', 'drink', 'family', 'gusto', 'happy', 'hello', 'hindi', 'house', 'how', 'i love you', 'inumin', 'itim', 'kahapon', 'kahel', 'kailan', 'kamusta', 'kotse', 'libro', 'mahal kita', 'masaya', 'ngayon', 'no', 'oo', 'orange', 'paano', 'pagod', 'pamilya', 'phone', 'read', 'saan', 'salamat', 'sino', 'sleep', 'sumulat', 'telepono', 'thankyou', 'tired', 'today', 'tomorrow', 'tubig', 'tulog', 'want', 'water', 'what', 'when', 'where', 'who', 'why', 'write', 'yes', 'yesterday']
📌 Total Classes: 60
💾 Saved processed dataset → C:\Users\JamJayDatuin\Documents\Machine Learning Projects\SignLanguageRecognition\dataset\ASL\processed
💾 Saved processed dataset → C:\Users\JamJayDatuin\Documents\Machine Learning Projects\SignLanguageRecognition\dataset\FSL\processed


c:\Users\JamJayDatuin\Documents\Machine Learning Projects\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_7 (Conv1D)               │ (None, 50, 64)         │        24,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv1d_7              │ (None, 50, 64)         │           256 │
│ (DepthwiseConv1D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_7 (MaxPooling1D)  │ (None, 25, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_14 (GRU)                    │ (None, 25, 128)        │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_15 (GRU)                    │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 60)             │         3,900 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 144,316 (563.73 KB)

 Trainable params: 144,316 (563.73 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
600/600 ━━━━━━━━━━━━━━━━━━━━ 68s 69ms/step - accuracy: 0.1188 - loss: 2.8937 - val_accuracy: 0.2275 - val_loss: 2.1814
Epoch 2/40
600/600 ━━━━━━━━━━━━━━━━━━━━ 33s 55ms/step - accuracy: 0.3141 - loss: 1.9112 - val_accuracy: 0.5529 - val_loss: 1.2638
Epoch 3/40
600/600 ━━━━━━━━━━━━━━━━━━━━ 33s 54ms/step - accuracy: 0.5712 - loss: 1.1163 - val_accuracy: 0.5796 - val_loss: 1.1426
Epoch 4/40
600/600 ━━━━━━━━━━━━━━━━━━━━ 34s 56ms/step - accuracy: 0.6958 - loss: 0.7633 - val_accuracy: 0.8075 - val_loss: 0.5247
Epoch 5/40
600/600 ━━━━━━━━━━━━━━━━━━━━ 44s 73ms/step - accuracy: 0.7360 - loss: 0.6399 - val_accuracy: 0.8121 - val_loss: 0.4608
Epoch 6/40
600/600 ━━━━━━━━━━━━━━━━━━━━ 36s 59ms/step - accuracy: 0.7931 - loss: 0.5094 - val_accuracy: 0.8594 - val_loss: 0.3409
Epoch 7/40
600/600 ━━━━━━━━━━━━━━━━━━━━ 30s 51ms/step - accuracy: 0.8051 - loss: 0.4721 - val_accuracy: 0.8773 - val_loss: 0.2900
Epoch 8/40
600/600 ━━━━━━━━━━━━━━━━━━━━ 32s 54ms/step - accuracy: 0.8477 - loss: 0.3916 - 

INFO:tensorflow:Assets written to: C:\Users\JAMJAY~1\AppData\Local\Temp\tmp3dkq6r9q\assets


Saved artifact at 'C:\Users\JAMJAY~1\AppData\Local\Temp\tmp3dkq6r9q'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 126), dtype=tf.float32, name='keras_tensor_747')
Output Type:
  TensorSpec(shape=(None, 60), dtype=tf.float32, name=None)
Captures:
  2199519073424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2201781123792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2201781121872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2199519072464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2201781122256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2199929724688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2199519079952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2202725231568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2202725230800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2202725236368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2

In [94]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from collections import deque, Counter

# =============================================
#              CONFIG
# =============================================
SEQ_LEN = 50
PREDICT_EVERY = 2
SMOOTH_WINDOW = 8

# =============================================
#              PATHS
# =============================================
MODEL_PATH = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\SignLanguageRecognition\models\Sign_Model.keras"
CLASSES_PATH = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\SignLanguageRecognition\models\classes.npy"

# =============================================
#              LOAD MODEL + LABELS
# =============================================
print("Loading Keras model...")
model = tf.keras.models.load_model(MODEL_PATH)

classes = np.load(CLASSES_PATH, allow_pickle=True)
print("Loaded classes:", classes)
num_classes = len(classes)

# =============================================
#              MEDIAPIPE SETUP
# =============================================
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5)

# =============================================
#              BUFFERS
# =============================================
sequence = deque(maxlen=SEQ_LEN)
predictions_smooth = deque(maxlen=SMOOTH_WINDOW)
recognized_sentence = []

cap = cv2.VideoCapture(0)
frame_count = 0

# =============================================
#              PREDICTION FUNCTION
# =============================================
def predict_label(window):
    array = np.array(window, dtype=np.float32).reshape(1, SEQ_LEN, 126)
    probs = model.predict(array, verbose=0)
    idx = np.argmax(probs)
    return classes[idx], float(probs[0][idx])


# =============================================
#         🔵 REAL-TIME LOOP
# =============================================
print("🎥 ASL+FSL Mixed Prediction Started...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    left = np.zeros(63)
    right = np.zeros(63)

    if not (results.multi_hand_landmarks and results.multi_handedness):
        cv2.putText(frame, "No Hands Detected", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        cv2.imshow("ASL+FSL Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    for i, hand in enumerate(results.multi_hand_landmarks):
        hand_label = results.multi_handedness[i].classification[0].label.lower()
        coords = []

        for lm in hand.landmark:
            coords.extend([lm.x, lm.y, lm.z])

        if hand_label == "left":
            left = np.array(coords)
        else:
            right = np.array(coords)

        mp_draw.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)

    sequence.append(np.concatenate([left, right]))

    frame_count += 1

    if len(sequence) == SEQ_LEN and frame_count % PREDICT_EVERY == 0:

        label, conf = predict_label(sequence)
        predictions_smooth.append(label)

        stable_label = Counter(predictions_smooth).most_common(1)[0][0]

        # PRINT TO TERMINAL ONLY
        print(f"Predicted: {label} | Confidence: {conf*100:.2f}% | Stable: {stable_label}")

        if len(recognized_sentence) == 0 or stable_label != recognized_sentence[-1]:
            recognized_sentence.append(stable_label)

    cv2.putText(frame, " ".join(recognized_sentence[-7:]),
                (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    cv2.imshow("ASL+FSL Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print("\nFinal Recognized Gloss Sequence:")
print(recognized_sentence)


Loading Keras model...
Loaded classes: ['ano' 'asul' 'bahay' 'bakit' 'basahin' 'black' 'blue' 'book' 'bukas'
 'car' 'drink' 'family' 'gusto' 'happy' 'hello' 'hindi' 'house' 'how'
 'i love you' 'inumin' 'itim' 'kahapon' 'kahel' 'kailan' 'kamusta' 'kotse'
 'libro' 'mahal kita' 'masaya' 'ngayon' 'no' 'oo' 'orange' 'paano' 'pagod'
 'pamilya' 'phone' 'read' 'saan' 'salamat' 'sino' 'sleep' 'sumulat'
 'telepono' 'thankyou' 'tired' 'today' 'tomorrow' 'tubig' 'tulog' 'want'
 'water' 'what' 'when' 'where' 'who' 'why' 'write' 'yes' 'yesterday']
🎥 ASL+FSL Mixed Prediction Started...
Predicted: yesterday | Confidence: 60.72% | Stable: yesterday
Predicted: tulog | Confidence: 44.25% | Stable: yesterday
Predicted: tulog | Confidence: 55.71% | Stable: tulog
Predicted: tulog | Confidence: 69.93% | Stable: tulog
Predicted: tulog | Confidence: 90.16% | Stable: tulog
Predicted: tulog | Confidence: 91.87% | Stable: tulog
Predicted: tulog | Confidence: 94.04% | Stable: tulog
Predicted: tulog | Confidence: 9